# GROUP BY

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

pd.options.display.float_format = '{:_.3f}'.format

In [2]:
df = sns.load_dataset('diamonds')
df

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.230,Ideal,E,SI2,61.500,55.000,326,3.950,3.980,2.430
1,0.210,Premium,E,SI1,59.800,61.000,326,3.890,3.840,2.310
2,0.230,Good,E,VS1,56.900,65.000,327,4.050,4.070,2.310
3,0.290,Premium,I,VS2,62.400,58.000,334,4.200,4.230,2.630
4,0.310,Good,J,SI2,63.300,58.000,335,4.340,4.350,2.750
...,...,...,...,...,...,...,...,...,...,...
53935,0.720,Ideal,D,SI1,60.800,57.000,2757,5.750,5.760,3.500
53936,0.720,Good,D,SI1,63.100,55.000,2757,5.690,5.750,3.610
53937,0.700,Very Good,D,SI1,62.800,60.000,2757,5.660,5.680,3.560
53938,0.860,Premium,H,SI2,61.000,58.000,2757,6.150,6.120,3.740


In [3]:
df.groupby('cut').mean()

,carat,depth,table,price,x,y,z
cut,,,,,,,
Ideal,0.703,61.709,55.952,3_457.542,5.507,5.520,3.401
Premium,0.892,61.265,58.746,4_584.258,5.974,5.945,3.647
Very Good,0.806,61.818,57.956,3_981.760,5.741,5.770,3.560
Good,0.849,62.366,58.695,3_928.864,5.839,5.851,3.640
Fair,1.046,64.042,59.054,4_358.758,6.247,6.183,3.983


In [4]:
df.groupby('cut')['price'].mean()

cut
Ideal       3_457.542
Premium     4_584.258
Very Good   3_981.760
Good        3_928.864
Fair        4_358.758
Name: price, dtype: float64

## Recorrer los grupos con un for

In [5]:
for key, group in df.groupby('cut'):
    price = group['price'].mean()
    print('Cut: {} \t Price: {}'.format(key,price))

Cut: Ideal 	 Price: 3457.541970210199
Cut: Premium 	 Price: 4584.2577042999055
Cut: Very Good 	 Price: 3981.7598907465654
Cut: Good 	 Price: 3928.864451691806
Cut: Fair 	 Price: 4358.757763975155


## Convirtiendo a DataFrame

In [6]:
# df.groupby(['cut','color'])['price'].mean().to_frame()
df.groupby(['cut','color'])['price'].aggregate(['min',np.mean,'max']).head(10)

min      mean    max
cut     color                      
Ideal   D      367 2_629.095  18693
        E      326 2_597.550  18729
        F      408 3_374.939  18780
        G      361 3_720.706  18806
        H      357 3_889.335  18760
        I      348 4_451.970  18779
        J      340 4_918.186  18508
Premium D      367 3_631.293  18575
        E      326 3_538.914  18477
        F      342 4_324.890  18791

### Aplicando distintas funciones a distintas columnas

In [7]:
def mean_kilo(x):
    return np.mean(x)/1000

In [8]:
dict_agg = {'carat':[min, max],'price':[np.mean, mean_kilo]}
dict_agg

{'carat': [<function min>, <function max>],
 'price': [<function numpy.mean(a, axis=None, dtype=None, out=None, keepdims=<no value>, *, where=<no value>)>,
  <function __main__.mean_kilo(x)>]}

In [9]:
# df.groupby(['cut','color']).apply(mean_kilo)
# df.groupby(['cut','color']).apply(lambda x : np.mean(x)/1000)
df.groupby(['cut','color']).aggregate(dict_agg).head(10)

carat           price          
                min   max      mean mean_kilo
cut     color                                
Ideal   D     0.200 2.750 2_629.095     2.629
        E     0.200 2.280 2_597.550     2.598
        F     0.230 2.450 3_374.939     3.375
        G     0.230 2.540 3_720.706     3.721
        H     0.230 3.500 3_889.335     3.889
        I     0.230 3.220 4_451.970     4.452
        J     0.230 3.010 4_918.186     4.918
Premium D     0.200 2.570 3_631.293     3.631
        E     0.200 3.050 3_538.914     3.539
        F     0.200 3.010 4_324.890     4.325

## Aplicando filtros

In [10]:
def filtro(x):
    return mean_kilo(x['price']) > 4

In [11]:
df.groupby('cut').filter(filtro)

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.210,Premium,E,SI1,59.800,61.000,326,3.890,3.840,2.310
3,0.290,Premium,I,VS2,62.400,58.000,334,4.200,4.230,2.630
8,0.220,Fair,E,VS2,65.100,61.000,337,3.870,3.780,2.490
12,0.220,Premium,F,SI1,60.400,61.000,342,3.880,3.840,2.330
14,0.200,Premium,E,SI2,60.200,62.000,345,3.790,3.750,2.270
...,...,...,...,...,...,...,...,...,...,...
53928,0.790,Premium,E,SI2,61.400,58.000,2756,6.030,5.960,3.680
53930,0.710,Premium,E,SI1,60.500,55.000,2756,5.790,5.740,3.490
53931,0.710,Premium,F,SI1,59.800,62.000,2756,5.740,5.730,3.430
53934,0.720,Premium,D,SI1,62.700,59.000,2757,5.690,5.730,3.580


In [12]:
df.groupby('cut').filter(filtro)['cut'].unique()

['Premium', 'Fair']
Categories (5, object): ['Ideal', 'Premium', 'Very Good', 'Good', 'Fair']

## Variables categóricas

In [15]:
df = sns.load_dataset('tips')
df['ones'] = 1
df

,total_bill,tip,sex,smoker,day,time,size,ones
0,16.990,1.010,Female,No,Sun,Dinner,2,1
1,10.340,1.660,Male,No,Sun,Dinner,3,1
2,21.010,3.500,Male,No,Sun,Dinner,3,1
3,23.680,3.310,Male,No,Sun,Dinner,2,1
4,24.590,3.610,Female,No,Sun,Dinner,4,1
...,...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3,1
240,27.180,2.000,Female,Yes,Sat,Dinner,2,1
241,22.670,2.000,Male,Yes,Sat,Dinner,2,1
242,17.820,1.750,Male,No,Sat,Dinner,2,1


In [16]:
df_g = df.groupby(['sex','smoker'])[['ones']].sum()
df_g

ones
sex    smoker      
Male   Yes       60
       No        97
Female Yes       33
       No        54

Transformando a porcentajes

In [20]:
df_g.groupby(level=0).apply(lambda x: x / x.sum() * 100)
# level = 0 para que agrupe por Male and Female

ones
sex    smoker       
Male   Yes    38.217
       No     61.783
Female Yes    37.931
       No     62.069

Vamos ahora a transformar variables numéricas en categóricas. Esto realizando intervalos.

In [21]:
pd.cut(df['total_bill'],bins = [3,18,35,60]).value_counts()

(3, 18]     127
(18, 35]    101
(35, 60]     16
Name: total_bill, dtype: int64

In [23]:
df['bin_total'] = pd.cut(df['total_bill'],bins = [3,18,35,60])
df

int64


,total_bill,tip,sex,smoker,day,time,size,ones,bin_total
0,16.990,1.010,Female,No,Sun,Dinner,2,1,"(3, 18]"
1,10.340,1.660,Male,No,Sun,Dinner,3,1,"(3, 18]"
2,21.010,3.500,Male,No,Sun,Dinner,3,1,"(18, 35]"
3,23.680,3.310,Male,No,Sun,Dinner,2,1,"(18, 35]"
4,24.590,3.610,Female,No,Sun,Dinner,4,1,"(18, 35]"
...,...,...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3,1,"(18, 35]"
240,27.180,2.000,Female,Yes,Sat,Dinner,2,1,"(18, 35]"
241,22.670,2.000,Male,Yes,Sat,Dinner,2,1,"(18, 35]"
242,17.820,1.750,Male,No,Sat,Dinner,2,1,"(3, 18]"


In [25]:
df.groupby(['time','bin_total'])[['ones']].count().groupby(level=0).apply(lambda x: x / x.sum() * 100)

ones
time   bin_total       
Lunch  (3, 18]   66.176
       (18, 35]  30.882
       (35, 60]   2.941
Dinner (3, 18]   46.591
       (18, 35]  45.455
       (35, 60]   7.955

# Pivot

In [28]:
df.groupby(['sex','time'])[['total_bill']].mean()

total_bill
sex    time              
Male   Lunch       18.048
       Dinner      21.461
Female Lunch       16.339
       Dinner      19.213

In [29]:
df_gp = df.groupby(['sex','time'])[['total_bill']].mean().reset_index()
df_gp

,sex,time,total_bill
0,Male,Lunch,18.048
1,Male,Dinner,21.461
2,Female,Lunch,16.339
3,Female,Dinner,19.213


In [30]:
df_gp.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.048,21.461
Female,16.339,19.213


![](https://pandas-docs.github.io/pandas-docs-travis/_images/reshaping_pivot.png)

Aplicando el *pivot* a todo el DF se obtiene el mismo resultado.

Internamente el pivot realiza el *mean* por defecto.

In [31]:
df.pivot_table(values='total_bill', index='sex', columns='time')

time,Lunch,Dinner
sex,,
Male,18.048,21.461
Female,16.339,19.213


In [32]:
df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc=np.median)

time,Lunch,Dinner
sex,,
Male,16.580,19.630
Female,13.420,17.190


In [33]:
df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc=[np.median,np.mean])

median          mean       
time    Lunch Dinner  Lunch Dinner
sex                               
Male   16.580 19.630 18.048 21.461
Female 13.420 17.190 16.339 19.213

In [36]:
df_pivot = df.pivot_table(values='total_bill', index='sex', columns='time', aggfunc=[np.median,np.mean])
df_pivot.unstack().reset_index()

,level_0,time,sex,0
0,median,Lunch,Male,16.580
1,median,Lunch,Female,13.420
2,median,Dinner,Male,19.630
3,median,Dinner,Female,17.190
4,mean,Lunch,Male,18.048
5,mean,Lunch,Female,16.339
6,mean,Dinner,Male,21.461
7,mean,Dinner,Female,19.213
